<a href="https://colab.research.google.com/github/tarabelo/PIAC-2526/blob/main/Computaci%C3%B3n%20cu%C3%A1ntica%20adiab%C3%A1tica%20y%20Quantum%20Annealing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Computación cuántica adiabática y Quantum Annealing**

### Contenidos

1. [Computación cuántica adiabática](#adiabatica)
1. [Quantum Annealing](#annealing)
1. [Variational Quantum Eigensolvers (VQE)](#vqe)


<a name="adiabatica"></a>

## **Computación cuántica adiabática**

Forma alternativa de computación cuántica basada en la evolución temporal de un Hamiltoniano y en el [teorema adiabático](https://en.wikipedia.org/wiki/Adiabatic_theorem)

#### Ecuación de Schrödinger

La evolución temporal de un sistema cuántico viene dada por la ecuación de Schrödinger:

$$i\hbar \frac{\partial}{\partial t}|\psi(t)\rangle = H(t)|\psi(t)\rangle$$

$H(t)$ es el Hamiltoniano del sistema en el instante $t$ y define la energía del mismo.

#### Teorema adiabático
Si un sistema cuántico está en un estado base (estado de menor energía o _ground state_) de un Hamiltoniano inicial y se le hace evolucionar muy lentamente hacia un Hamiltoniano final, este estado inicial evolucionará al estado base del Hamiltoniano final, tras un tiempo suficientemente largo.

Supongamos que tenemos un Hamiltoniano $H_c$ (_Hamiltoniano de coste_) cuyo estado base codifica la solución de nuestro problema.

Partimos de un Hamiltoniano $H_0$ cuyo estado base es facil de preparar y queremos llegar, en un tiempo $T$, al estado base de $H_c$. Hacemos evolucionar el sistema desde $t=0$ hasta $t=T$ como sigue:

$$H(t) = A(t)H_0 + B(t)H_c$$

donde las funciones $A$ y $B$ verifican: $A(0) = B(T) = 1$ y $A(T) = B(0) = 0$.

Un ejemplo de estas funciones son

$$
A(t) =  \left(1-\frac{t}{T}\right)\\
B(t) = \frac{t}{T}
$$


Si $t$ cambia muy lentamente, el sistema pasa de $H(0) = H_0$ a $H(T) = H_c$ manteniéndose en el estado base.

Midiendo el estado del sistema en $t=T$ obtenemos el estado $|\psi\rangle$ que minimiza $\langle\psi|H_c|\psi\rangle$ y que es la solución del problema.

En [Aharonov et al. 2007](https://doi.org/10.1137/080734479) se demostró que la computación cuántica adiabática es equivalente a la computación cuántica basada en puertas.

#### Tiempo de evolución

El teorema adiabático establece que para que $H(t)$ se mantenga en el estado base, $T$ tiene que ser inversamente proporcional al cuadrado del *spectral gap* (minima diferencia de energía entre el estado base y el primer estado excitado de $H(t)$ con $t\in[0,T]$).

Encontrar el valor del *spectral gap* es muy complejo. Además, $T$ podría llegar a ser muy grande. El *Quantum annealing* es una implementación práctica de la computación cuántica adiabática.




<a name="annealing"></a>

## **Quantum annealing**

Es un modelo similar a la computación cuántica adiabática, con ciertas limitaciones:

- El Hamiltoniano final tiene que ser un Hamiltoniano Ising:  

$$
H_c = -\sum_{i,j=0}^{n-1} J_{ij}Z_iZ_j - \sum_{i=0}^{n-1} h_{i}Z_i
$$

- No se mantiene la evolución adiabática $\implies$ el estado final podría no ser el estado base de $H_c$

**Estado inicial $H_0$**

Se suele usar como estado inicial el siguiente:

$$
H_0 = -\sum_{i=0}^{n-1} X_i
$$

Este estado se denomina _mixing Hamiltonian_ y su estado base es $|\psi_0\rangle = |+\rangle^{\otimes n}$

**Evolución del Hamiltoniano**

La evolución viene dada por:

$$
H(t) = A(t)H_0 + B(t)H_c = -A(t)\sum_{i=0}^{n-1} X_i - B(t)\sum_{i,j=0}^{n-1} J_{ij}Z_iZ_j - B(t)\sum_{i=0}^{n-1} h_{i}Z_i
$$


### Quantum annealers
Computadores cuánticos que resuelven problemas de optimización mediante Quantum annealing.

- [D-Wave Systems](https://www.dwavesys.com/): Primera empresa en vender quantum annealers
  - El último modelo dispone de más de 7000 cúbits
  - Suite Ocean: incluye diferentes paquetes, entre otros:
    - dwave-system: para enviar problemas a los sistemas cuánticos de D-Wave (incluyendo a través del servicio en la nube [Leap](https://cloud.dwavesys.com/)).
    - dimod: para definir estructuras de datos y modelos matemáticos de diferentes tipos de problemas.
    - dwave-networkx: integración con NetworkX para grafos.
    - dwave-neal: simulador clásico basado en [simulated annealing](https://en.wikipedia.org/wiki/Simulated_annealing).
    
  - Tipo de problemas: optimización de portfolios, optimización de rutas y gestión de flotas, optimización de líneas de producción, búsqueda de similaridad entre moléculas, etc.




In [4]:
!pip install dwave-ocean-sdk dwave-neal

Para obtener la versión instalada:

In [ ]:
!dwave --version

DWave ofrecía un acceso gratuito general (limitado) a sus sistemas, pero lo canceló en enedro de 2025:

  - El programa [Leap Quantum LaunchPad](https://www.dwavequantum.com/quantum-launchpad/) proporciona acceso por 3 meses a usuarios cualificados.

Al darse de alta, se obtiene una API key que se configura ejecutando:

In [ ]:
!dwave config create

**Ejemplo**

<center><img src="https://drive.google.com/uc?export=view&id=1pHIFMBSBWWciVhG1ll_SAKibigkHOiKz" alt="Malla Ising 3 nodos" width="350"  /></center>

El Hamiltoniano Ising de esta malla es:
$$
H = -\sum_{(i,j) \in E} J_{ij} Z_iZ_j - \sum_j h_j Z_j = Z_0Z_1 - 3Z_1Z_2
$$

Es decir, $J_{01}=-1$, $J_{12} = 3$, $h_j=0$.

Este problema se puede expresar usando [dimod](https://docs.dwavequantum.com/en/latest/ocean/api_ref_dimod/), la biblioteca base de D-Wave Ocean para representar y resolver distintos modelos de problemas con y sin restricciones, como los siguientes:

 - Modelos cuadráticos binarios (QBM), como QUBO o Ising.
 - Modelos cuadráticos con restricciones
 - Modelos discretos cuadráticos

In [ ]:
from dimod import BinaryQuadraticModel, SPIN
J = {(0,1):-1, (1,2):3}
h = {}

# Expresamos el grafo simple como un modelo binario cuadrático
# SPIN: las variables toman valores {-1,1} (Ising)
problem = BinaryQuadraticModel(h, J, 0.0, SPIN)
print("El problema a resolver es:")
print(problem)

DWave ofrecía un acceso gratuito general (limitado) a sus sistemas, pero lo canceló en 2025:

  - El programa [Leap Quantum LaunchPad](https://www.dwavequantum.com/quantum-launchpad/) proporciona acceso por 3 meses a usuarios cualificados.

Para resolver el problema en un sistema de DWave, haríamos:

In [ ]:
from dwave.system import DWaveSampler
from dwave.system import EmbeddingComposite

sampler = EmbeddingComposite(DWaveSampler())

# Hacemos 10 ejecuciones
result = sampler.sample(problem, num_reads=10)
print("Las soluciones obtenidas son")
print(result)

Principales funciones:

  - [DWaveSampler](https://docs.dwavequantum.com/en/latest/ocean/api_ref_system/samplers.html#dwavesampler): permite usar un sistema de DWave como un sampler en el código.
  - [EmbeddingComposite](https://docs.dwavequantum.com/en/latest/ocean/api_ref_system/composites.html#dwave.system.composites.EmbeddingComposite): permite mapear el problema en la configuraciñón física del sistema DWave.

### Solución con dimod y simulación

Dimod incluye un resolutor por fuerza bruta, que prueba todas las posibles combinaciones

In [ ]:
from dimod import ExactSolver
solver = ExactSolver()
solution = solver.sample(problem)
print(solution)

Podemos usar el simulador basado en annealing simulado

In [3]:
# neal: simulador de annealing de DWave
from neal import SimulatedAnnealingSampler

# Usamos un simulador clásico en vez del D-WaveSampler
sampler = SimulatedAnnealingSampler()

# Ejecutamos 10 repeticiones
result = sampler.sample(problem, num_reads=10)

print("\nLas soluciones obtenidas son:")
print(result)


ModuleNotFoundError: No module named 'neal'



---



---



## **Ejercicio 3**

Para el grafo de la imagen, expresa el algoritmo MAXCUT en formato QUBO, exprésalo en dimod y resuelvelo de forma exacta y con Annealing simulado

<center><img src="https://drive.google.com/uc?export=view&id=1t37PWgnsVWRkX16wcbmFSPZNZ1D0dUOS" alt="Ejemplo MAX-CUT" width="700"  /></center>




---



---



#### Uso de _dwave-networkx_

El módulo dwave-networkx permite resolver el problema anterior de forma simple, definiendo el grafo mediante la librería _networkx_ de Python


In [6]:
import numpy as np
import networkx as nx  # Librería para manejar grafos
import dwave_networkx as dnx
from neal import SimulatedAnnealingSampler

# Ejemplo de grafo con 5 nodos
nnodes = 5
G = nx.Graph()
# Añade nodos y aristas
G.add_nodes_from(np.arange(0,nnodes,1))
edges = [(0,1,3),(1,2,2),(1,3,4),(2,4,1),(2,3,7),(3,4,5)]
G.add_weighted_edges_from(edges)


# Obtener un lado del  usando annealing simulado
cut = dnx.weighted_maximum_cut(G, sampler=SimulatedAnnealingSampler())

# Calcular aristas cortadas y peso total
cut_edges = [(int(u), int(v), G[u][v]['weight']) for u, v in G.edges() if (u in cut) ^ (v in cut)]
cut_value = sum(w for _, _, w in cut_edges)

print("Un lado del corte:", cut)
print("Aristas cortadas:", cut_edges)
print("Peso total del corte:", cut_value)

Un lado del corte: {np.int64(0), np.int64(3)}
Aristas cortadas: [(0, 1, 3), (1, 3, 4), (2, 3, 7), (3, 4, 5)]
Peso total del corte: 19


### Problemas binarios cuadráticos con restricciones

Supongamos un problema como este:
\begin{align*}
\text{Minimizar}\quad & -5x_0 +3 x_1 - 2x_2 \\
\text{sujeto a}\quad & x_0+x_1 \le 1, \\
& 3x_0-x_1+3x_2 \le 4,\\
& x_j \in \{0,1\}, j=0,1,2
\end{align*}

Usando dmod lo expresamos como un problema CQM (_Constrained cuadratic Model_) de la siguiente forma:

In [12]:
from dimod import Binary, ConstrainedQuadraticModel
x0, x1, x2 = Binary('x0'), Binary('x1'), Binary('x2')

cqm = ConstrainedQuadraticModel()
cqm.set_objective(-5*x0+3*x1-2*x2)
cqm.add_constraint(x0+x1 <= 1, 'Primera restricción')
cqm.add_constraint(3*x0-x1+3*x2 <= 4, 'Segunda restricción')

print(cqm)

Constrained quadratic model: 3 variables, 2 constraints, 8 biases

Objective
  -5*Binary('x0') + 3*Binary('x1') - 2*Binary('x2')

Constraints
  Primera restricción: Binary('x0') + Binary('x1') <= 1.0
  Segunda restricción: 3*Binary('x0') - Binary('x1') + 3*Binary('x2') <= 4.0

Bounds



In [22]:
from dimod import ExactCQMSolver

# Resolver con ExactCQMSolver
solver = ExactCQMSolver()
sampleset = solver.sample_cqm(cqm)

print(sampleset)

solutions = sampleset.filter(lambda s: s.is_feasible)

print(solutions)

# Mejor solución factible
best = sampleset.first
print("Mejor solución:", best)
print("Valor objetivo:", best.energy)

  x0 x1 x2 energy num_oc. is_sat. is_fea.
6  1  0  1   -7.0       1 arra... np.F...
2  1  0  0   -5.0       1 arra... np.T...
7  1  1  1   -4.0       1 arra... np.F...
3  1  1  0   -2.0       1 arra... np.F...
4  0  0  1   -2.0       1 arra... np.T...
0  0  0  0    0.0       1 arra... np.T...
5  0  1  1    1.0       1 arra... np.T...
1  0  1  0    3.0       1 arra... np.T...
['INTEGER', 8 rows, 8 samples, 3 variables]
  x0 x1 x2 energy num_oc. is_sat. is_fea.
2  1  0  0   -5.0       1 arra... np.T...
3  0  0  1   -2.0       1 arra... np.T...
0  0  0  0    0.0       1 arra... np.T...
4  0  1  1    1.0       1 arra... np.T...
1  0  1  0    3.0       1 arra... np.T...
['INTEGER', 5 rows, 5 samples, 3 variables]
Mejor solución: Sample(sample={'x0': np.int64(1), 'x1': np.int64(0), 'x2': np.int64(1)}, energy=np.float64(-7.0), num_occurrences=np.int64(1), is_satisfied=array([ True, False]), is_feasible=np.False_)
Valor objetivo: -7.0


In [23]:
print(solutions.first)

Sample(sample={'x0': np.int64(1), 'x1': np.int64(0), 'x2': np.int64(0)}, energy=np.float64(-5.0), num_occurrences=np.int64(1), is_satisfied=array([ True,  True]), is_feasible=np.True_)
